In [2]:
import nltk
import pandas as pd
import pyarabic.araby as araby
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import word_tokenize
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import arabicstopwords.arabicstopwords as stp
from farasa.stemmer import FarasaStemmer
nltk.download('punkt')
#Loading data from csv into data drame
quoran = pd.read_csv("qurantexttanzil.csv")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AURES\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
quoran = pd.read_csv("qurantexttanzil.csv")
soura = quoran["text"]
b=[]
for a in soura:
    b.append(araby.strip_diacritics(a))
quoran['diacriticless'] = b

In [8]:
tokenized = []
for i in b:
    print(wordpunct_tokenize(i))
    tokenized.append(wordpunct_tokenize(i))
# adding a new column in the data frame for "tokenized"
quoran["tokenized"] = tokenized

['بسم', 'الله', 'الرحمن', 'الرحيم']
['الحمد', 'لله', 'رب', 'العالمين']
['الرحمن', 'الرحيم']
['مالك', 'يوم', 'الدين']
['إياك', 'نعبد', 'وإياك', 'نستعين']
['اهدنا', 'الصراط', 'المستقيم']
['صراط', 'الذين', 'أنعمت', 'عليهم', 'غير', 'المغضوب', 'عليهم', 'ولا', 'الضالين']
['بسم', 'الله', 'الرحمن', 'الرحيم', 'الم']
['ذلك', 'الكتاب', 'لا', 'ريب', 'فيه', 'هدى', 'للمتقين']
['الذين', 'يؤمنون', 'بالغيب', 'ويقيمون', 'الصلاة', 'ومما', 'رزقناهم', 'ينفقون']
['والذين', 'يؤمنون', 'بما', 'أنزل', 'إليك', 'وما', 'أنزل', 'من', 'قبلك', 'وبالآخرة', 'هم', 'يوقنون']
['أولئك', 'على', 'هدى', 'من', 'ربهم', 'وأولئك', 'هم', 'المفلحون']
['إن', 'الذين', 'كفروا', 'سواء', 'عليهم', 'أأنذرتهم', 'أم', 'لم', 'تنذرهم', 'لا', 'يؤمنون']
['ختم', 'الله', 'على', 'قلوبهم', 'وعلى', 'سمعهم', 'وعلى', 'أبصارهم', 'غشاوة', 'ولهم', 'عذاب', 'عظيم']
['ومن', 'الناس', 'من', 'يقول', 'آمنا', 'بالله', 'وباليوم', 'الآخر', 'وما', 'هم', 'بمؤمنين']
['يخادعون', 'الله', 'والذين', 'آمنوا', 'وما', 'يخدعون', 'إلا', 'أنفسهم', 'وما', 'يشعرون']
['في', 'قلوب

In [9]:
stopwords = ['فما','إنهم','وعلى','وأولئك','ومما','إليك','بما','والذين','فيه','لا','ذلك','ولا','عليهم','غير','الذين','إياك','إن','إنك','وإياك','سواء','ولهم','معكم','كانوا','مثلهم', 'كمثل','فلما','حوله','فهم','وأنتم','كنتم','ولن','تحتها','قبل','وهم','مثلا','أنه','وأما','كثيرا','إليه','جميعا','إني','قال','قالوا','ونحن','كلها','فلما','قلنا','وكان','فتكونا','عنها','بعضكم','لبعض','فإما','معكم',
         'وهي','أولم','كانت','سواء','هو','قال','لا','لكل','الذين','هذا','إنما','لك','يا','إذا','أن','عند','لها','الذي','لنا','في','لو','ذلك','إني','لكم','كذلك','قد','إليه','له','قالوا','ما','لهم','من','هل','وإن','مما','وهم','فيها','فيه','إذ','به','على','إن','حتى','بل','عليكم','أنا','هي','الأولى','لن','هم','حين','ليس','كل','بين','نحن','هؤلاء','ومن','إنه','بما','فإن','إنهم','ولا','عليهم','أم'
          ,'عن','أو','إنا','أدراك','إلا','وما','إلى','ثم','تكونوا','أول','وأن','يكون','قل','كان','عليكم','منا','منكم','لعلكم','وأنا','أيها','وإنما','كانت','إنها','عني','عليها','علينا','فإنها','منا','عنده','وليس','وإنا','أيها','و','وقد','بني','وكانوا','جاء','أحيانا','قبل','ولما','معهم' 
  ]
def stop_words_remover(word) :
        if not stp.is_stop(word): 
           if not word in  stopwords: return  ''.join(word)
        else: return None 

In [10]:
stemmer = FarasaStemmer()
def stem(ayah) :
     return stemmer.stem(ayah)

x= ",".join(quoran.diacriticless)
x=stem(x)
# adding a new column in the data frame for "stem_text"
quoran["stem_text"]=x.split(",")

In [20]:
quran1 = [nltk.word_tokenize(i) for i in quoran['stem_text']]
quoran["stemed_and_removed"]= [
    [stop_words_remover(word) for word in ayah ] for ayah in quran1]
corpus = quoran["stemed_and_removed"].apply(lambda x: [item for item in x if item != None])
quoran["clean_text"]=corpus

In [21]:
embedding_size = 100
window_size = 15
min_word = 0
down_sampling = 1e-2

In [22]:
from gensim.models import Word2Vec
wv_model=Word2Vec(corpus,
                  vector_size=embedding_size,
                  window=window_size,
                  min_count=min_word,
                  workers=4,
                  seed=42,
                  epochs=30,
                  sg=1,
                  sample=down_sampling
)

In [23]:
semantically_similar_words_wv = {words: [item[0] for item in wv_model.wv.most_similar([words], topn=5)]
                  for words in ['الله']}

for k,v in semantically_similar_words_wv.items():
    print(k+":"+str(v))
# we can improve our search by indicating positive words and negative ones
wv_model.wv.most_similar(positive=['مريم','عيسى'], negative=['موسى'], topn=1)

الله:['متوكل', 'يلتكم', 'آذيتمونا', 'استنبأ', 'مزمل']


[('وجيه', 0.6351176500320435)]

In [24]:
print(wv_model.wv.similarity(w1='مريم', w2='عيسى'))

0.8310492


In [25]:
s1 = 'قال الله'
s2 = ' محمد '
#Compute cosine distance between two keys, the closer to one the smaller the angle the greater the match
#while 0 means the verctors are at 90 deg and they are dissimilar
distance = wv_model.wv.n_similarity(s1.split(), s2.split())
print(distance)

0.46296605


In [26]:
def findSimilarSentence(arg):
    output = [{
        'sentence': '',
        'score': 0, 
        'index': 0,
    }]
    for ayah in quoran['diacriticless']:
        output.append({'sentence': ayah, 'score':  wv_model.wv.n_similarity(arg.lower().split(), ayah.lower().split()), 'index': quoran[quoran['diacriticless'] == ayah].index[0]})
    return output

temp = findSimilarSentence('إنك ميت وإنهم ميتون')
temp.sort(key=lambda x: x['score'], reverse=True)
temp[:10]

[{'sentence': 'إنك ميت وإنهم ميتون', 'score': 1.0, 'index': 4095},
 {'sentence': 'وهو الذي يرسل الرياح بشرا بين يدي رحمته حتى إذا أقلت سحابا ثقالا سقناه لبلد ميت فأنزلنا به الماء فأخرجنا به من كل الثمرات كذلك نخرج الموتى لعلكم تذكرون',
  'score': 0.9999316,
  'index': 1038},
 {'sentence': 'والله الذي أرسل الرياح فتثير سحابا فسقناه إلى بلد ميت فأحيينا به الأرض بعد موتها كذلك النشور',
  'score': 0.789081,
  'index': 3690},
 {'sentence': 'أفما نحن بميتين', 'score': 0.74220234, 'index': 3862},
 {'sentence': 'لنحيي به بلدة ميتا ونسقيه مما خلقنا أنعاما وأناسي كثيرا',
  'score': 0.68101645,
  'index': 2936},
 {'sentence': 'رزقا للعباد وأحيينا به بلدة ميتا كذلك الخروج',
  'score': 0.68101645,
  'index': 4653},
 {'sentence': 'هو الذي يريكم البرق خوفا وطمعا وينشئ السحاب الثقال',
  'score': 0.65952903,
  'index': 1746},
 {'sentence': 'وإن يروا كسفا من السماء ساقطا يقولوا سحاب مركوم',
  'score': 0.6367621,
  'index': 4784},
 {'sentence': 'كيف تكفرون بالله وكنتم أمواتا فأحياكم ثم يميتكم ثم يحييكم ث